In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import threading
from queue import Queue
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
watermark_DIR = "watermark\\WaterM.png"

In [2]:
def toprint(df):
    columns_to_analyze = df.columns[1:]  # Start from the 2nd column (Column2)

    max_values = df[columns_to_analyze].max()
    min_values = df[columns_to_analyze].min()
    mean_values = df[columns_to_analyze].mean()

    print("Maximum values:")
    print(max_values)
    print("\nMinimum values:")
    print(min_values)
    print("\nMean values:")
    print(mean_values)
    
def imperceptibility(images_DIR):


    def calculate_metrics(original_image, water, queue):

        psnr_value = psnr(original_image, water)
        ssim_value = ssim(original_image, water, channel_axis=-1)
        correlation = np.corrcoef(original_image.flatten(), water.flatten())[0, 1]

        queue.put((image_path, psnr_value, ssim_value, correlation))

        return psnr_value, ssim_value, correlation



    def get_image_names(folder_path):
    
        image_names = []
        for filename in os.listdir(folder_path):
            image_names.append(filename)
        return image_names


    
    
    image_names = get_image_names(images_DIR)
    data = {'Image Name': [], 'PSNR': [], 'SSIM': [], 'CC': []}
    queue = Queue() 
    threads = [] 


    for imgname in image_names:
        image_path = images_DIR + imgname
        image = cv2.imread(image_path)
        water = cv2.imread(watermark_DIR)

        # create a thread for each image pair and pass the queue as an argument
        thread = threading.Thread(target=calculate_metrics, args=(image, water, queue))
        threads.append(thread) # add the thread to the list
        thread.start() # start the thread

    for thread in threads:
        thread.join() # wait for all threads to finish

    while not queue.empty(): # get the results from the queue
        image_path, psnr_value, ssim_value, correlation = queue.get()

        data['Image Name'].append(os.path.basename(image_path))
        data['PSNR'].append(psnr_value)
        data['SSIM'].append(ssim_value)
        data['CC'].append(correlation)



    df = pd.DataFrame(data)


    #print(df)
    return df

In [5]:
%%time

images_DIR = "extracted\\"
df=imperceptibility(images_DIR)
toprint(df)


Maximum values:
PSNR    41.027525
SSIM     0.987170
CC       0.999813
dtype: float64

Minimum values:
PSNR    25.068264
SSIM     0.977933
CC       0.991896
dtype: float64

Mean values:
PSNR    39.060178
SSIM     0.984119
CC       0.999246
dtype: float64
CPU times: total: 250 ms
Wall time: 53.3 ms


In [5]:
df.to_csv('fileWATER.csv')

In [6]:
%%time
images_DIR = "attack\\extracted\\rotation\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    10.926574
SSIM     0.785872
CC       0.876710
dtype: float64

Minimum values:
PSNR    4.969273
SSIM    0.146381
CC      0.126755
dtype: float64

Mean values:
PSNR    9.726286
SSIM    0.653622
CC      0.715612
dtype: float64
CPU times: total: 39 s
Wall time: 5.23 s


In [7]:
%%time
images_DIR = "attack\\extracted\\downscale\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    11.393319
SSIM     0.692258
CC       0.988097
dtype: float64

Minimum values:
PSNR    6.398190
SSIM    0.192082
CC      0.257612
dtype: float64

Mean values:
PSNR    10.710475
SSIM     0.613132
CC       0.928766
dtype: float64
CPU times: total: 38 s
Wall time: 5.12 s


In [8]:
%%time
images_DIR = "attack\\extracted\\upscale\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    15.906599
SSIM     0.899607
CC       0.997665
dtype: float64

Minimum values:
PSNR    7.319070
SSIM    0.311918
CC      0.393419
dtype: float64

Mean values:
PSNR    14.799247
SSIM     0.839595
CC       0.978215
dtype: float64
CPU times: total: 38.4 s
Wall time: 5.15 s


In [9]:
%%time
images_DIR = "attack\\extracted\\gaussBlurr\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    10.471232
SSIM     0.618321
CC       0.985513
dtype: float64

Minimum values:
PSNR    6.877980
SSIM    0.229339
CC      0.299578
dtype: float64

Mean values:
PSNR    10.058873
SSIM     0.560444
CC       0.949507
dtype: float64
CPU times: total: 38.8 s
Wall time: 5.23 s


In [10]:
%%time
images_DIR = "attack\\extracted\\saltpepper\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    20.504006
SSIM     0.897294
CC       0.975630
dtype: float64

Minimum values:
PSNR    7.164110
SSIM    0.328779
CC      0.424311
dtype: float64

Mean values:
PSNR    17.648494
SSIM     0.846110
CC       0.945860
dtype: float64
CPU times: total: 38.5 s
Wall time: 5.12 s


In [6]:
%%time
#50%
images_DIR = "attack\\extracted\\jpeg50\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    14.646523
SSIM     0.777799
CC       0.917236
dtype: float64

Minimum values:
PSNR    13.525978
SSIM     0.737587
CC       0.874440
dtype: float64

Mean values:
PSNR    13.990090
SSIM     0.755407
CC       0.894842
dtype: float64
CPU times: total: 297 ms
Wall time: 42 ms


In [12]:
%%time
#95% default
images_DIR = "attack\\extracted\\jpeg\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    34.521770
SSIM     0.970510
CC       0.999136
dtype: float64

Minimum values:
PSNR    7.099818
SSIM    0.320819
CC      0.412302
dtype: float64

Mean values:
PSNR    25.566869
SSIM     0.915742
CC       0.980247
dtype: float64
CPU times: total: 38.2 s
Wall time: 5.18 s


In [3]:
%%time
#30%
images_DIR = "attack\\extracted\\jpeg30\\"
df=imperceptibility(images_DIR)
toprint(df)


Maximum values:
PSNR    10.769013
SSIM     0.618786
CC       0.777135
dtype: float64

Minimum values:
PSNR    9.269633
SSIM    0.471354
CC      0.704611
dtype: float64

Mean values:
PSNR    10.163690
SSIM     0.561979
CC       0.741825
dtype: float64
CPU times: total: 1.12 s
Wall time: 101 ms


In [4]:

%%time
#40%
images_DIR = "attack\\extracted\\jpeg40\\"
df=imperceptibility(images_DIR)
toprint(df)


Maximum values:
PSNR    12.849209
SSIM     0.719683
CC       0.877991
dtype: float64

Minimum values:
PSNR    11.982337
SSIM     0.670211
CC       0.818624
dtype: float64

Mean values:
PSNR    12.402728
SSIM     0.689031
CC       0.846461
dtype: float64
CPU times: total: 688 ms
Wall time: 97 ms


In [5]:

%%time
#60%
images_DIR = "attack\\extracted\\jpeg60\\"
df=imperceptibility(images_DIR)
toprint(df)


Maximum values:
PSNR    16.570882
SSIM     0.827184
CC       0.945969
dtype: float64

Minimum values:
PSNR    14.622681
SSIM     0.762977
CC       0.901942
dtype: float64

Mean values:
PSNR    15.721656
SSIM     0.804361
CC       0.929360
dtype: float64
CPU times: total: 719 ms
Wall time: 99 ms


In [6]:

%%time
#70%
images_DIR = "attack\\extracted\\jpeg70\\"
df=imperceptibility(images_DIR)
toprint(df)


Maximum values:
PSNR    19.181898
SSIM     0.865441
CC       0.969102
dtype: float64

Minimum values:
PSNR    16.134127
SSIM     0.802668
CC       0.932788
dtype: float64

Mean values:
PSNR    17.833328
SSIM     0.840158
CC       0.956077
dtype: float64
CPU times: total: 734 ms
Wall time: 100 ms


In [7]:

%%time
#80%
images_DIR = "attack\\extracted\\jpeg80\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    21.618743
SSIM     0.898234
CC       0.982698
dtype: float64

Minimum values:
PSNR    17.583858
SSIM     0.829974
CC       0.953353
dtype: float64

Mean values:
PSNR    19.963099
SSIM     0.868373
CC       0.972898
dtype: float64
CPU times: total: 609 ms
Wall time: 100 ms


In [3]:

%%time

images_DIR = "attack\\extracted\\bright05\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    11.315497
SSIM     0.661126
CC       0.999454
dtype: float64

Minimum values:
PSNR    11.010891
SSIM     0.648183
CC       0.991394
dtype: float64

Mean values:
PSNR    11.284157
SSIM     0.658718
CC       0.998871
dtype: float64
CPU times: total: 1.5 s
Wall time: 162 ms


In [4]:
%%time

images_DIR = "attack\\extracted\\bright06\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    13.255015
SSIM     0.789651
CC       0.999621
dtype: float64

Minimum values:
PSNR    12.758414
SSIM     0.775123
CC       0.991556
dtype: float64

Mean values:
PSNR    13.205817
SSIM     0.786664
CC       0.999048
dtype: float64
CPU times: total: 672 ms
Wall time: 101 ms


In [5]:
%%time

images_DIR = "attack\\extracted\\bright07\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    15.772902
SSIM     0.884180
CC       0.999672
dtype: float64

Minimum values:
PSNR    14.939028
SSIM     0.870119
CC       0.991657
dtype: float64

Mean values:
PSNR    15.674387
SSIM     0.880955
CC       0.999084
dtype: float64
CPU times: total: 641 ms
Wall time: 103 ms


In [6]:
%%time

images_DIR = "attack\\extracted\\bright08\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    19.266971
SSIM     0.945281
CC       0.999702
dtype: float64

Minimum values:
PSNR    17.741710
SSIM     0.932592
CC       0.991670
dtype: float64

Mean values:
PSNR    19.119844
SSIM     0.942300
CC       0.999118
dtype: float64
CPU times: total: 844 ms
Wall time: 100 ms


In [7]:
%%time

images_DIR = "attack\\extracted\\bright09\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    25.202541
SSIM     0.976629
CC       0.999710
dtype: float64

Minimum values:
PSNR    21.464494
SSIM     0.964808
CC       0.991750
dtype: float64

Mean values:
PSNR    24.841332
SSIM     0.973849
CC       0.999137
dtype: float64
CPU times: total: 750 ms
Wall time: 104 ms


In [8]:
%%time

images_DIR = "attack\\extracted\\bright11\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    29.944520
SSIM     0.975831
CC       0.998413
dtype: float64

Minimum values:
PSNR    25.106523
SSIM     0.956428
CC       0.992004
dtype: float64

Mean values:
PSNR    29.272953
SSIM     0.970324
CC       0.997614
dtype: float64
CPU times: total: 688 ms
Wall time: 110 ms


In [9]:
%%time

images_DIR = "attack\\extracted\\bright12\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    26.877262
SSIM     0.964636
CC       0.996682
dtype: float64

Minimum values:
PSNR    13.947314
SSIM     0.794619
CC       0.886584
dtype: float64

Mean values:
PSNR    22.871136
SSIM     0.915597
CC       0.977620
dtype: float64
CPU times: total: 688 ms
Wall time: 100 ms


In [10]:
%%time

images_DIR = "attack\\extracted\\bright13\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    24.184520
SSIM     0.951934
CC       0.993549
dtype: float64

Minimum values:
PSNR    8.868193
SSIM    0.546938
CC      0.624683
dtype: float64

Mean values:
PSNR    16.757990
SSIM     0.799753
CC       0.911636
dtype: float64
CPU times: total: 703 ms
Wall time: 98.3 ms


In [11]:
%%time

images_DIR = "attack\\extracted\\bright14\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    26.587281
SSIM     0.941689
CC       0.994818
dtype: float64

Minimum values:
PSNR    7.769502
SSIM    0.313805
CC      0.511706
dtype: float64

Mean values:
PSNR    14.159920
SSIM     0.678815
CC       0.844152
dtype: float64
CPU times: total: 828 ms
Wall time: 111 ms


In [12]:
%%time

images_DIR = "attack\\extracted\\bright15\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    24.820982
SSIM     0.933599
CC       0.991357
dtype: float64

Minimum values:
PSNR    6.749012
SSIM    0.127481
CC      0.474007
dtype: float64

Mean values:
PSNR    12.386751
SSIM     0.593277
CC       0.786079
dtype: float64
CPU times: total: 766 ms
Wall time: 101 ms


In [3]:

images_DIR = "attack\\extracted\\crop25UL\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    12.298336
SSIM     0.745813
CC       0.844542
dtype: float64

Minimum values:
PSNR    12.070299
SSIM     0.736605
CC       0.836257
dtype: float64

Mean values:
PSNR    12.254204
SSIM     0.741710
CC       0.842831
dtype: float64


In [4]:
images_DIR = "attack\\extracted\\crop25W\\"
df=imperceptibility(images_DIR)
toprint(df)

Maximum values:
PSNR    41.027525
SSIM     0.987170
CC       0.999813
dtype: float64

Minimum values:
PSNR    25.068285
SSIM     0.977933
CC       0.991896
dtype: float64

Mean values:
PSNR    39.059917
SSIM     0.984119
CC       0.999246
dtype: float64


In [13]:
#df.to_csv('fileWATER.csv')